In [1]:
from pathlib import Path
from typing import Annotated, Union

import typer
from peft import AutoPeftModelForCausalLM, PeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    PreTrainedTokenizerFast,
)

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ModelType = Union[PreTrainedModel, PeftModelForCausalLM]
TokenizerType = Union[PreTrainedTokenizer, PreTrainedTokenizerFast]

def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def load_model_and_tokenizer(model_dir: Union[str, Path]) -> tuple[ModelType, TokenizerType]:
    model_dir = _resolve_path(model_dir)
    if (model_dir / 'adapter_config.json').exists():
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=True, device_map='auto'
        )
        tokenizer_dir = model.peft_config['default'].base_model_name_or_path
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=True, device_map='auto'
        )
        tokenizer_dir = model_dir
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_dir, trust_remote_code=True
    )
    return model, tokenizer

In [3]:
model_dir = "output/api-cloze-test"

model, tokenizer = load_model_and_tokenizer(model_dir)

Loading checkpoint shards: 100%|██████████| 7/7 [02:53<00:00, 24.82s/it]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [4]:
import pandas as pd
from tools.common_utils import highlight_diff, read_jsonl

json_file_path = 'data/dev_cloze_test.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,conversations
0,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
1,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
2,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
3,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
4,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."


In [6]:
from tools.standard_name_utils import get_standard_api, optimize_api_chain, optimize_parameters

standard_api = get_standard_api("./data/api_定义.json")

data_stock = pd.read_excel('./raw_data/标准名.xlsx',sheet_name='股票标准名')
data_fund = pd.read_excel('./raw_data/标准名.xlsx',sheet_name='基金标准名')
fund_standard_name = data_stock['标准股票名称'].to_list()
stock_standard_name = data_fund['标准基金名称'].to_list()

In [8]:
print(standard_api[:5])
print(fund_standard_name[:5])
print(stock_standard_name[:5])

['条件选基_查询代码', '条件选基_查询类型', '条件选基_查询规模', '条件选基_查询晨星评级', '条件选基_查询蚂蚁评级']
['万马股份', '汉王科技', '珠江啤酒', '联络互动', '奇正藏药']
['华夏成长证券投资基金', '华夏大盘精选证券投资基金A类', '易方达天天理财货币市场基金R类', '华夏优势增长混合型证券投资基金', '泰达宏利信用合利定期开放债券型证券投资基金A类']


In [10]:
from random import random

top_p=0.7 
temperature=0.95

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    single_user_row = row["conversations"][0]
    single_assis_row = row["conversations"][1]
    
    input = single_user_row["content"]
    output = single_assis_row["content"]
    
    
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    # optimized_resp = optimize_api_chain(response, standard_api)
    # optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    optimized_resp = response
    
    total_eval_count += 1
    if optimized_resp == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, optimized_resp))
        print(optimized_resp)
        print()
    
    if total_eval_count == 20:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

-----data index-----
14
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，api的调用链chain已经给出，你需要用适合的参数和结果替换chain中的MOCK。query是：通过投资索菱的成交量和成交额，我想推测出本股的平均成交价格，并了解今天的主力资金流向。 
 query中提到的产品标准名可能是：海富通上证投资级可转债及可交换债券交易型开放式指数证券投资基金、汇添富中证港股通高股息投资交易型开放式指数证券投资基金、鹏华港股通中证香港中小企业投资主题指数证券投资基金(LOF)、华泰柏瑞中证港股通高股息投资交易型开放式指数证券投资基金(QDII)、易方达ESG责任投资股票型发起式证券投资基金、财通中证香港红利等权投资指数型证券投资基金C类、财通中证香港红利等权投资指数型证券投资基金A类、融通债券投资基金C类、华泰柏瑞中证港股通高股息投资交易型开放式指数证券投资基金发起式联接基金C类、华泰柏瑞中证港股通高股息投资交易型开放式指数证券投资基金发起式联接基金A类、融通新蓝筹证券投资基金、融通新蓝筹证券投资基金、海富通精选证券投资基金、创金合信ESG责任投资股票型发起式证券投资基金C类、创金合信ESG责任投资股票型发起式证券投资基金A类、融通行业景气证券投资基金、融通蓝筹成长证券投资基金、融通债券投资基金A/B类、融通债券投资基金A/B类、创金合信气候变化责任投资股票型发起式证券投资基金C类、创金合信气候变化责任投资股票型发起式证券投资基金A类、融通通穗债券型证券投资基金、融通通益混合型证券投资基金、融通通玺债券型证券投资基金、融通通润债券型证券投资基金、融通通弘债券型证券投资基金、融通通安债券型证券投资基金、融通通和债券型证券投资基金、融通通优债券型证券投资基金、融通增裕债券型证券投资基金、融通增悦债券型证券投资基金、融通增利债券型证券投资基金、融通增丰债券型证券投资基金、海富通收益增长证券投资基金、泰康港股通中证香港银行投资指数型发起式证券投资基金C类、泰康港股通中证香港银行投资指数型发起式证券投资基金A类、易方达中证香港证券投资主题交易型开放式指数证券投资基金、鹏华港股通中证香港银行投资指数证券投资基金(LOF)C类、鹏华港股通中证香港银行投资指数证券投资基金(LOF)A类、融